In [1]:
# using the 39 phone set proposed in (Lee & Hon, 1989)
# Table 3. Mapping from 61 classes to 39 classes, as proposed by Lee and Hon, (Lee & Hon,
# 1989). The phones in the left column are folded into the labels of the right column. The
# remaining phones are left intact.}
!pip install trax
import trax
import trax.layers as tl
import random as rnd
import tensorflow as tf
import json
from tensorflow.keras.layers import Flatten
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
!apt-get update && apt-get install -y python3-opencv
!pip install opencv-python
import cv2
import itertools




from trax.fastmath import numpy as fastnp
import numpy as np
import os


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8183 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [239 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [14.6 kB]
Fetched 8644 kB in 2s (5130 kB/s)                           
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core dbus dbus-user-session
  dconf-gsettings-backend dconf-service dmsetup gdal-data glib-networking
  glib-networking-common glib-networking-services gsettings-desktop-schemas
  gstreamer1.0-plugins-base gtk-update-icon-cache i965-va-driver
  intel-media-va-driver

In [2]:

def collect_video_files(data_type, resp, root_dir):
    if resp == 'lipspeakers':
        speakerDir = root_dir+'/lipspeakers'
    elif resp == 'volunteers':
        speakerDir = root_dir+'/volunteers'
    else:
        speakerDir = root_dir

    fileName = f'{data_type}_dict.json'
    data = open(speakerDir+os.sep+fileName).read()
    data_dict = json.loads(data)

    with open(f'{speakerDir}/data_phn.txt', 'r') as f:
        lines = f.readlines()
    video_list = [l.strip() for l in lines]

    return data_dict, video_list, speakerDir




video_path = os.path.join('/kaggle/input/volunt')
spkr_type = 'volunteerss'

lip_data_dict, lip_video_list, lip_data_dir = collect_video_files('data', 'volunteers',video_path)


rnd.shuffle(lip_video_list)
lip_video_list=list(set(lip_video_list))
train_size=int(len(lip_video_list)*.9)
train_list=lip_video_list[:train_size]

val_list=lip_video_list[train_size:]

In [3]:
len(train_list),len(val_list)

(4672, 520)

In [4]:
phoneme_set_39_list = [
    'iy', 'ih', 'eh', 'ae', 'ah', 'uw', 'uh', 'aa', 'ey', 'ay', 'oy', 'aw', 'ow',
    'l', 'r', 'y', 'w', 'er', 'm', 'n', 'ng', 'ch', 'jh', 'dh', 'b', 'd', 'dx',
    'g', 'p', 't', 'k', 'z', 'v', 'f', 'th', 's', 'sh', 'hh', 'sil'
]
def get_vocabs(phoneme_set_39_list):
    phonem_nunmber={'<pad>':0}
    number_phonem={0:'<pad>'}
    for phonem in phoneme_set_39_list:
        phonem_nunmber[phonem]=len(phonem_nunmber)
        number_phonem[len(number_phonem)]=phonem
    return phonem_nunmber,number_phonem

phonem_nunmber,number_phonem=get_vocabs(phoneme_set_39_list)
len(phonem_nunmber),len(number_phonem)

(40, 40)

In [5]:
mobile = tf.keras.applications.mobilenet.MobileNet(include_top=True, 
                                                weights='imagenet')
x = mobile.layers[-7].output
output=Flatten(name='flatten')(x)
output=Dense(1024,activation='relu')(output)
model = Model(inputs=mobile.input, outputs=output)


17225924/17225924 [==============================] - 0s 0us/step


In [37]:
def gemerte_date(data,data_labels,data_dir,phonem_nunmber,model):
    data_size=len(data)
    idx=0
    while True:
        if idx>=data_size:
            idx=0


        video=data[idx]
        spk,vdo = video.split('_')[:2]
        image_name = vdo.split('.')[0]+'_'

        list_IDs_temp = sorted([image_file for image_file in list(data_labels) if image_name in image_file])
        list_IDs_temp=sorted([image_file for image_file in list_IDs_temp if image_file in os.listdir(data_dir+'/'+spk)])

        idx+=1
        X = np.zeros((len(list_IDs_temp),1024))
        Y = []
        for i, ID in enumerate(list_IDs_temp):

            im = cv2.imread(data_dir+'/'+spk+'/'+ID)
            im=cv2.resize(im,(224,224))
            im=list(model.predict(np.expand_dims(im,axis=0),verbose=0)[0])
            X[i] = im
            Y.append(phonem_nunmber[data_labels[ID]])
        Y=np.array(Y)
        yield ((X,Y))

In [7]:
X,Y=next(gemerte_date(train_list,lip_data_dict,lip_data_dir,phonem_nunmber,model))
X.shape,Y.shape

((24, 1024), (24,))

In [38]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [8,8,8,8,8,8,8,8]
# Create the generators.
train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gemerte_date(train_list,lip_data_dict,lip_data_dir,phonem_nunmber,model))

val_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gemerte_date(val_list,lip_data_dict,lip_data_dir,phonem_nunmber,model))


train_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(train_batch_stream)
val_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(val_batch_stream)



In [11]:
X,Y,M=next(train_batch_stream)
X.shape,Y.shape,M.shape

((8, 64, 1024), (8, 64), (8, 64))

In [29]:
# UNQ_C1
# GRADED FUNCTION
def input_encoder_fn(n_encoder_layers):
    """ Input encoder runs on the input sentence and creates
    activations that will be the keys and values for attention.
    
    Args:
        input_vocab_size: int: vocab size of the input
        d_model: int:  depth of embedding (n_units in the LSTM cell)
        n_encoder_layers: int: number of LSTM layers in the encoder
    Returns:
        tl.Serial: The input encoder
    """
    
    # create a serial network
    input_encoder = tl.Serial( 

        # feed the embeddings to the LSTM layers. It is a stack of n_encoder_layers LSTM layers
        [tl.LSTM(n_units=1024) for _ in range(n_encoder_layers)]
        ### END CODE HERE ###
    )

    return input_encoder

In [30]:
# UNQ_C2
# GRADED FUNCTION
def pre_attention_decoder_fn(mode, target_vocab_size, d_model):
    """ Pre-attention decoder runs on the targets and creates
    activations that are used as queries in attention.
    
    Args:
        mode: str: 'train' or 'eval'
        target_vocab_size: int: vocab size of the target
        d_model: int:  depth of embedding (n_units in the LSTM cell)
    Returns:
        tl.Serial: The pre-attention decoder
    """
    
    # create a serial network
    pre_attention_decoder = tl.Serial(
        
        ### START CODE HERE ###
        # shift right to insert start-of-sentence token and implement
        # teacher forcing during training
        tl.ShiftRight(),

        # run an embedding layer to convert tokens to vectors
        tl.Embedding(target_vocab_size,d_model),

        # feed to an LSTM layer
        tl.LSTM(d_model)
        ### END CODE HERE ###
    )
    
    return pre_attention_decoder

In [31]:
# UNQ_C3
# GRADED FUNCTION
def prepare_attention_input(encoder_activations, decoder_activations, inputs):
    """Prepare queries, keys, values and mask for attention.
    
    Args:
        encoder_activations fastnp.array(batch_size, padded_input_length, d_model): output from the input encoder
        decoder_activations fastnp.array(batch_size, padded_input_length, d_model): output from the pre-attention decoder
        inputs fastnp.array(batch_size, padded_input_length): input tokens
    
    Returns:
        queries, keys, values and mask for attention.
    """
    
    ### START CODE HERE ###
    
    # set the keys and values to the encoder activations
    keys = encoder_activations
    values = encoder_activations

    
    # set the queries to the decoder activations
    queries = decoder_activations
    
    # generate the mask to distinguish real tokens from padding
    # hint: inputs is positive for real tokens and 0 where they are padding
    mask =np.sum(inputs,axis=2)
    mask=mask!=0
    ### END CODE HERE ###
    
    # add axes to the mask for attention heads and decoder length.
    mask = fastnp.reshape(mask, (mask.shape[0], 1, 1, mask.shape[1]))
    
    # broadcast so mask shape is [batch size, attention heads, decoder-len, encoder-len].
    # note: for this assignment, attention heads is set to 1.
    mask = mask + fastnp.zeros((1, 1, decoder_activations.shape[1], 1))
    
        
    
    return queries, keys, values, mask

In [32]:
# UNQ_C4
# GRADED FUNCTION
def NMTAttn(input_vocab_size=33300,
            target_vocab_size=len(phonem_nunmber),
            d_model=1024,
            n_encoder_layers=6,
            n_decoder_layers=6,
            n_attention_heads=8,
            attention_dropout=0.1,
            mode='train'):
    """Returns an LSTM sequence-to-sequence model with attention.

    The input to the model is a pair (input tokens, target tokens), e.g.,
    an English sentence (tokenized) and its translation into German (tokenized).

    Args:
    input_vocab_size: int: vocab size of the input
    target_vocab_size: int: vocab size of the target
    d_model: int:  depth of embedding (n_units in the LSTM cell)
    n_encoder_layers: int: number of LSTM layers in the encoder
    n_decoder_layers: int: number of LSTM layers in the decoder after attention
    n_attention_heads: int: number of attention heads
    attention_dropout: float, dropout for the attention layer
    mode: str: 'train', 'eval' or 'predict', predict mode is for fast inference

    Returns:
    An LSTM sequence-to-sequence model with attention.
    """

    ### START CODE HERE ###
    
    # Step 0: call the helper function to create layers for the input encoder
    input_encoder = input_encoder_fn(n_encoder_layers)

    # Step 0: call the helper function to create layers for the pre-attention decoder
    pre_attention_decoder = pre_attention_decoder_fn(mode, target_vocab_size, d_model)

    # Step 1: create a serial network
    model = tl.Serial( 
        
      # Step 2: copy input tokens and target tokens as they will be needed later.
      tl.Select([0,1,0,1]),
        
      # Step 3: run input encoder on the input and pre-attention decoder the target.
      tl.Parallel(input_encoder,pre_attention_decoder),
        
      # Step 4: prepare queries, keys, values and mask for attention.
      tl.Fn('PrepareAttentionInput', prepare_attention_input, n_out=4),
        
      # Step 5: run the AttentionQKV layer
      # nest it inside a Residual layer to add to the pre-attention decoder activations(i.e. queries)
      tl.Residual(tl.AttentionQKV(d_model, n_heads=n_attention_heads, dropout=attention_dropout, mode=mode)),
      
      # Step 6: drop attention mask (i.e. index = None
      tl.Select([0,2]),
        
      # Step 7: run the rest of_de the RNN decoder
      [tl.LSTM(d_model) for _ in range(n_decoder_layers)],
        
      # Step 8: prepare output by making it the right size
      tl.Dense(target_vocab_size),   
      # Step 9: Log-softmax for output
      tl.LogSoftmax()
    )
    
    ### END CODE HERE
    
    return model

In [33]:
# UNQ_C5
# GRADED PART
from trax.supervised import training
def train_task_function(train_batch_stream):
    """Returns a trax.training.TrainTask object.

    Args:
    train_batch_stream generator: labeled data generator

    Returns:
    A trax.training.TrainTask object.
    """
    return training.TrainTask(

        ### START CODE HERE

        # use the train batch stream as labeled (
        labeled_data= train_batch_stream,

        # use the cross entropy loss
        loss_layer= tl.CrossEntropyLoss(),

        # use the Adam optimizer with learning rate of 0.01
        optimizer= trax.optimizers.Adam(0.001),

        # use the `trax.lr.warmup_and_rsqrt_decay` as th.01e learning rate schedule
        # have 1000 warmup steps with a max value of 0.01
        

        # have a checkpoint every 10 steps
        n_steps_per_checkpoint= 100

        ### END CODE HERE
    )

In [39]:
train_task = train_task_function(train_batch_stream)

In [40]:

eval_task = training.EvalTask(
    
    ## use the eval batch stream as labeled data
    labeled_data=val_batch_stream,
    
    ## use the cross entropy loss and accuracy as metrics
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)


In [41]:
# define the output directory
output_dir = 'output_dir/'


# define the training loop
training_loop = training.Loop(NMTAttn(mode='train'),
                              train_task,
                              eval_tasks=[eval_task],
                              output_dir=output_dir)

In [ ]:
n_step=int(len(train_list)/8)*100
print(n_step)
training_loop.run(n_step)


In [33]:
X.shape

(1, 16, 1024)

400

In [34]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working/output_dir'
remove_folder_contents(folder_path)
os.rmdir(r'/kaggle/working/output_dir')

In [5]:
(batch,timestep)
embedding
(batch,timestep,n_dimension)
(batch,timestep,w,h,channel)
(batch,timestep,1024)

NameError: name 'batch' is not defined